In [10]:
# Importing necessary libraries
from surprise import SVD, SVDpp, NMF
from surprise import Dataset
from surprise.model_selection import GridSearchCV
from surprise.model_selection import cross_validate

from pandas import DataFrame

### Знайдемо найкращі параметри для SVD

Завантажемо дані.

In [24]:
# Load the movielens dataset
data = Dataset.load_builtin("ml-100k")

# What's inside?
data_df = DataFrame(data.raw_ratings, columns=["user", "item", "rate", "time"])
print(data_df.head())


  user item  rate       time
0  196  242   3.0  881250949
1  186  302   3.0  891717742
2   22  377   1.0  878887116
3  244   51   2.0  880606923
4  166  346   1.0  886397596


**Наша цільова змінна - "rate"**

Вгадаємо параметри.

In [25]:
# Define a parameter grid
param_grid = {
    "n_epochs": [5, 10, 20],
    "n_factors": [50, 100, 200],
    "lr_all": [0.002, 0.005],
    "reg_all": [0.4, 0.6],
}

# Define a GridSearchCV instance
gs = GridSearchCV(SVD, param_grid, measures=["rmse", "mae"], cv=5)

# Fit the grid search instance
gs.fit(data)

# Best RMSE score
print(gs.best_score["rmse"])

# Combination of parameters that gave the best RMSE score
print(gs.best_params["rmse"])

0.9567816528585036
{'n_epochs': 20, 'n_factors': 50, 'lr_all': 0.005, 'reg_all': 0.4}


Бачимо, що найкраща (найменша) середньоквадратична похибка для такої моделі дають параметри

{'n_epochs': 20, 'n_factors': 50, 'lr_all': 0.005, 'reg_all': 0.4}


Переконаємося у цьому, явно подивившись результати **крос-валідацій**:

In [21]:
from itertools import product

# Define the parameters
n_epochs = [5, 10, 20]
n_factors = [50, 100, 200]
lr_all = [0.002, 0.005]
reg_all = [0.4, 0.6]

# Create a list of dictionaries, each containing a unique combination of parameters
param_grid = [dict(zip(['n_epochs', 'n_factors', 'lr_all', 'reg_all'], values)) for values in product(n_epochs, n_factors, lr_all, reg_all)]

# Iterate over all parameters
for params in param_grid:
    algo = SVD(**params)

    # Perform cross validation
    results = cross_validate(algo, data, measures=["rmse", "mae"], cv=5, verbose=True)

    # Print RMSE and MAE scores
    print(f"Parameters {params}\nRMSE: ", results['test_rmse'].mean())
    print("MAE: ", results["test_mae"].mean())

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9864  0.9908  0.9946  0.9879  0.9923  0.9904  0.0029  
MAE (testset)     0.7958  0.8001  0.8020  0.7959  0.8010  0.7990  0.0026  
Fit time          0.17    0.16    0.16    0.16    0.16    0.16    0.00    
Test time         0.20    0.11    0.11    0.20    0.11    0.15    0.04    
Parameters {'n_epochs': 5, 'n_factors': 50, 'lr_all': 0.002, 'reg_all': 0.4}
RMSE:  0.9903860456649156
MAE:  0.7989576659310178
Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9955  0.9984  0.9975  0.9940  1.0009  0.9973  0.0024  
MAE (testset)     0.8082  0.8087  0.8099  0.8062  0.8097  0.8085  0.0013  
Fit time          0.14    0.16    0.15    0.16    0.16    0.15    0.01    
Test time         0.11    0.20    0.11    0.11    0.19    0.14    0.04    
Parameters {'n

Дійсно, параметри 

{'n_epochs': 20, 'n_factors': 50, 'lr_all': 0.005, 'reg_all': 0.4}

дають найменші середньоквадратичну і середню похибки

RMSE:  0.9556055389869924,
MAE:  0.7652664014848375

Це означає, що модель вгадує оцінку юзера для даного кіно з похибкою трохи мешою за 1 зірку з 5ти. 

Оскільки це порядка 20%, це величезна похибка і модель треба покращувати далі, або використовувати інші. 

### SVDpp

In [29]:
# Define a parameter grid
param_grid = {
    "n_epochs": [20],
    "n_factors": [50, 100],
    "lr_all": [0.002, 0.005],
    "reg_all": [0.4, 0.6],
}

# Define a GridSearchCV instance
gs = GridSearchCV(SVDpp, param_grid, measures=["rmse", "mae"], cv=5)

# Fit the grid search instance
gs.fit(data)

# Best RMSE score
print(gs.best_score["rmse"])

# Combination of parameters that gave the best RMSE score
print(gs.best_params["rmse"])

0.9564452137376623
{'n_epochs': 20, 'n_factors': 50, 'lr_all': 0.005, 'reg_all': 0.4}


Цей алгоритм більш ресурсо-затратний, і результат також гірший. 

### NMF

In [32]:
# Define a parameter grid
param_grid = {
    "n_epochs": [5, 10],
    "n_factors": [50, 100],
    "reg_pu": [0.4, 0.6],
    "reg_qi": [0.4, 0.6],
}
# Define a GridSearchCV instance
gs = GridSearchCV(NMF, param_grid, measures=["rmse", "mae"], cv=5)

# Fit the grid search instance
gs.fit(data)

# Best RMSE score
print(gs.best_score["rmse"])

# Combination of parameters that gave the best RMSE score
print(gs.best_params["rmse"])

0.9646472845108172
{'n_epochs': 10, 'n_factors': 100, 'reg_pu': 0.4, 'reg_qi': 0.4}


Ми отримали ще гірший результат.

### Висновок 

Наразі найкращий результат на основі випадкових здогадок дала **SVD** з

**{'n_epochs': 20, 'n_factors': 50, 'lr_all': 0.005, 'reg_all': 0.4}**

і похибкою **~0.957**.

Такий результат недостатній на мою думку, щоб використовувати таку модель в реальному житті. 

Можливо, кращу модель скоріше вийде підібрати варіюючи різні варіанти параметрів використовуючи SVD (у порівнянні з SVDpp та NFM), якщо починати з випадкових здогадок. 

Оскільки в моделі базових параметри чотири, то навіть в межах цих порядків, здогадка для яких була взята з документації surprise, навіть з кроком 0.1 процес знаходження найменшої середньоквадратичної похибки займе багато розрахунків і довгий час. 

Щоб пришвидшити процес, можливо потрібно більш свідомо підійти до вибору початкових здогадок, особливо для SVD++.
